# Deterministic Inputs, Noisy “And” gate model (DINA)

This notebook will show you how to train and use the DINA.
First, we will show how to get the data (here we use Math1 from math2015 as the dataset).
Then we will show how to train a DINA and perform the parameters persistence.
At last, we will show how to load the parameters from the file and evaluate on the test dataset.

The script version could be found in [DINA.py](DINA.ipynb)

## Data Preparation

Before we process the data, we need to first acquire the dataset which is shown in [prepare_dataset.ipynb](prepare_dataset.ipynb)

In [1]:
# Load the data from files
import numpy as np
import json

q_m = np.loadtxt("../../../data/math2015/Math1/q_m.csv", dtype=int, delimiter=',')
prob_num, know_num = q_m.shape[0], q_m.shape[1]

# training data
with open("../../../data/math2015/Math1/train_data.json", encoding='utf-8') as file:
    train_set = json.load(file)
stu_num = max([x['user_id'] for x in train_set]) + 1
R = -1 * np.ones(shape=(stu_num, prob_num))
for log in train_set:
    R[log['user_id'], log['item_id']] = log['score']

# testing data
with open("../../../data/math2015/Math1/test_data.json", encoding='utf-8') as file:
    test_set = json.load(file)

FileNotFoundError: ../../../data/math2015/Math1/q_m.csv not found.

In [2]:
print(train_set[0], test_set[0])

{'user_id': 0, 'item_id': 5, 'score': 1.0} {'user_id': 0, 'item_id': 8, 'score': 1.0}


In [3]:
len(train_set), len(test_set)

(67344, 16836)

## Training and Persistence

In [4]:
import logging
logging.getLogger().setLevel(logging.INFO)

In [5]:
from EduCDM import EMDINA as DINA

cdm = DINA(R, q_m, stu_num, prob_num, know_num, skip_value=-1)

cdm.train(epoch=2, epsilon=1e-3)
cdm.save("dina.params")

INFO:root:save parameters to dina.params


## Loading and Testing

In [6]:
cdm.load("dina.params")
rmse, mae = cdm.eval(test_set)
print("RMSE: %.6f, MAE: %.6f" % (rmse, mae))

INFO:root:load parameters from dina.params
evaluating: 100%|█████████████████████████████████████████████████████████████| 16836/16836 [00:00<00:00, 67652.66it/s]


RMSE: 0.425828, MAE: 0.353139


## Incremental Training

In [7]:
new_data = [{'user_id': 0, 'item_id': 0, 'score': 1.0}, {'user_id': 1, 'item_id': 2, 'score': 0.0}]
cdm.inc_train(new_data, epoch=2, epsilon=1e-3)

## Evaluate User's State

In [8]:
stu_rec = np.array([0, 1, -1, 0, -1, 0, 1, 1, 0, 1, 0, 1, 0, -1, -1, -1, -1, 0, 1, -1])
dia_id, dia_state = cdm.transform(stu_rec)
print("id of user's state is %d, state is " % dia_id + str(dia_state))

id of user's state is 63, state is [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1.]


In [9]:
# To see the relation between dia_id and dia_state
dia_state == cdm.all_states[dia_id]

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])